In [6]:
import os
import re
from scipy.io import loadmat

def load_hyperspectral_data(dataset_name):
    """
    Load hyperspectral data and ground truth from the Data folder using pattern matching.
    
    Args:
        dataset_name (str): Base name of the dataset (e.g., 'Pavia', 'Salinas')
        
    Returns:
        tuple: (hyperspectral_data, ground_truth)
        
    Raises:
        FileNotFoundError: If matching data or ground truth files cannot be found
        ValueError: If multiple matching files are found for the same pattern
    """
    data_dir = 'Data'
    
    data_pattern = re.compile(f"{dataset_name}.*\.mat$", re.IGNORECASE)
    gt_pattern = re.compile(f"{dataset_name}.*_gt.*\.mat$", re.IGNORECASE)
    
    data_files = [f for f in os.listdir(data_dir) if data_pattern.match(f)]
    gt_files = [f for f in os.listdir(data_dir) if gt_pattern.match(f)]
    
    data_files = [f for f in data_files if f not in gt_files]
    
    if not data_files or not gt_files:
        raise FileNotFoundError(f"Could not find matching files for {dataset_name} in {data_dir}")
    if len(data_files) > 1 or len(gt_files) > 1:
        raise ValueError(f"Multiple matching files found for {dataset_name}")
        
    data_path = os.path.join(data_dir, data_files[0])
    gt_path = os.path.join(data_dir, gt_files[0])
    
    data_mat = loadmat(data_path)
    gt_mat = loadmat(gt_path)
    
    hyperspectral_data = next(v for k, v in data_mat.items() if not k.startswith('__'))
    ground_truth = next(v for k, v in gt_mat.items() if not k.startswith('__'))
    
    return hyperspectral_data, ground_truth

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

def visualize_ground_truth(ground_truth):
    """
    Visualize ground truth labels for Salinas dataset with distinct colors and legend.
    Background (0) is shown in black.
    """
    # Define class names
    class_names = {
        0: 'Background',
        1: 'Brocoli_green_weeds_1',
        2: 'Brocoli_green_weeds_2',
        3: 'Fallow',
        4: 'Fallow_rough_plow',
        5: 'Fallow_smooth',
        6: 'Stubble',
        7: 'Celery',
        8: 'Grapes_untrained',
        9: 'Soil_vinyard_develop',
        10: 'Corn_senesced_green_weeds',
        11: 'Lettuce_romaine_4wk',
        12: 'Lettuce_romaine_5wk',
        13: 'Lettuce_romaine_6wk',
        14: 'Lettuce_romaine_7wk',
        15: 'Vinyard_untrained',
        16: 'Vinyard_vertical_trellis'
    }
    
    # Create colormap with black for background
    colors = ['black'] + list(plt.cm.tab20(np.linspace(0, 1, 20))[:16])
    cmap = ListedColormap(colors)
    
    plt.figure(figsize=(12, 8))
    im = plt.imshow(ground_truth, cmap=cmap)
    plt.title('Salinas Ground Truth Labels')
    
    # Create legend (excluding background)
    legend_elements = [plt.Rectangle((0,0),1,1, facecolor=colors[i]) 
                      for i in range(len(class_names))]
    plt.legend(legend_elements, 
              [f'{k}: {v}' for k, v in class_names.items()],
              bbox_to_anchor=(1.05, 1), 
              loc='upper left',
              borderaxespad=0.)
    
    plt.tight_layout()
    plt.show()

data, ground_truth = load_hyperspectral_data('Pavia')    
visualize_ground_truth(ground_truth)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

def plot_spectral_signatures(data, labels, title="Average Spectral Signatures"):
    """
    Plot average spectral signatures for each class in the Pavia dataset with enhanced styling.
    """
    class_names = {
        1: 'Water',
        2: 'Trees',
        3: 'Asphalt',
        4: 'Self-Blocking Bricks',
        5: 'Bitumen',
        6: 'Tiles',
        7: 'Shadows',
        8: 'Meadows',
        9: 'Bare Soil'
    }
    
    # Set style for professional visualization
    plt.style.use('ggplot')
    
    # Create figure with specific size ratio
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Reshape data
    h, w, bands = data.shape
    reshaped_data = data.reshape(-1, bands)
    flat_labels = labels.reshape(-1)
    
    # Color palette for distinct classes
    colors = plt.cm.Set2(np.linspace(0, 1, len(class_names)))
    
    # Plot each class
    for idx, label in enumerate(sorted(class_names.keys())):
        mask = flat_labels == label
        avg_spectrum = np.mean(reshaped_data[mask], axis=0)
        std_spectrum = np.std(reshaped_data[mask], axis=0)
        
        # Plot mean line and confidence interval
        ax.plot(range(bands), avg_spectrum, 
                label=f'{class_names[label]}',
                color=colors[idx],
                linewidth=2)
        ax.fill_between(range(bands), 
                       avg_spectrum - std_spectrum,
                       avg_spectrum + std_spectrum,
                       color=colors[idx],
                       alpha=0.1)
    
    # Customize appearance
    ax.set_xlabel('Spectral Band', fontsize=12, fontweight='bold')
    ax.set_ylabel('Reflectance', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    
    # Customize grid
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Customize legend
    ax.legend(bbox_to_anchor=(1.05, 1),
             loc='upper left',
             borderaxespad=0,
             fontsize=10,
             frameon=True,
             title='Surface Types',
             title_fontsize=12)
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    plt.show()

# Load and visualize Pavia dataset
data, labels = load_hyperspectral_data('Pavia')
plot_spectral_signatures(data, labels, "Spectral Signatures of Urban Surface Types - Pavia Dataset")

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def plot_tsne_visualization(data, labels, title="t-SNE Visualization of Hyperspectral Data"):
   """
   Create t-SNE visualization of hyperspectral data with class-based coloring for Indian Pines dataset.
   """
   class_names = {
       1: 'Alfalfa',
       2: 'Corn-notill', 
       3: 'Corn-mintill',
       4: 'Corn',
       5: 'Grass-pasture',
       6: 'Grass-trees',
       7: 'Grass-pasture-mowed',
       8: 'Hay-windrowed',
       9: 'Oats',
       10: 'Soybean-notill',
       11: 'Soybean-mintill',
       12: 'Soybean-clean',
       13: 'Wheat',
       14: 'Woods',
       15: 'Buildings-Grass-Trees-Drives',
       16: 'Stone-Steel-Towers'
   }
   
   # Reshape data and remove background pixels
   h, w, bands = data.shape
   data_reshaped = data.reshape(-1, bands)
   labels_flat = labels.reshape(-1)
   
   # Remove background pixels (label 0)
   mask = labels_flat != 0
   data_filtered = data_reshaped[mask]
   labels_filtered = labels_flat[mask]
   
   print("Performing t-SNE dimensionality reduction...")
   tsne = TSNE(
    n_components=2,
    perplexity=40,  # Increased from default 30
    learning_rate='auto',  # Adaptive learning rate
    n_iter=3000,  # Increased iterations
    init='pca',  # PCA initialization
    random_state=42
    )
   embedded = tsne.fit_transform(data_filtered)
   
   fig, ax = plt.subplots(figsize=(14, 10), facecolor='white')
   ax.set_facecolor('white')
   
   # Using a colormap suitable for many categories
   scatter = ax.scatter(embedded[:, 0], embedded[:, 1], 
                       c=labels_filtered,
                       cmap='tab20',
                       alpha=0.7,
                       s=8) 
   
   ax.set_title("Spectral Distribution of Agricultural Classes - Indian Pines Dataset", 
                fontsize=14, fontweight='bold', pad=20)
   ax.set_xlabel('First t-SNE Component', fontsize=12)
   ax.set_ylabel('Second t-SNE Component', fontsize=12)
   ax.grid(True, linestyle='--', alpha=0.3, color='gray')
   
   # Create legend with class names and sample counts
   legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                                markerfacecolor=scatter.cmap(scatter.norm(label)), 
                                label=f'{class_names[label]} (n={np.sum(labels_filtered == label)})', 
                                markersize=8)
                     for label in sorted(class_names.keys())]
   
   ax.legend(handles=legend_elements,
            bbox_to_anchor=(1.05, 1),
            loc='upper left',
            borderaxespad=0,
            title='Agricultural Classes',
            title_fontsize=12)
   
   plt.tight_layout()
   plt.show()

# Load and visualize Indian Pines dataset
data, labels = load_hyperspectral_data('Indian_pines')
plot_tsne_visualization(data, labels)

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

def plot_datasets(save_path=None):
   datasets = ['Salinas', 'Indian_pines', 'Pavia']
   fig, axes = plt.subplots(1, 3, figsize=(12, 3), gridspec_kw={'wspace': -0.2})

   for idx, dataset in enumerate(datasets):
       data, gt = load_hyperspectral_data(dataset)
       n_classes = len(np.unique(gt))
       
       colors = plt.cm.Spectral(np.linspace(0, 1, n_classes))
       colors[0] = [0.9, 0.9, 0.9, 1]  # Light gray for background
       custom_cmap = ListedColormap(colors)
       
       im = axes[idx].imshow(gt, cmap=custom_cmap, interpolation='nearest')
       axes[idx].set_title(dataset.replace('_', ' '), pad=10, 
                         fontsize=10, fontweight='bold', 
                         fontfamily='Times New Roman')
       axes[idx].axis('off')
       
       divider = make_axes_locatable(axes[idx])
       cax = divider.append_axes("right", size="5%", pad=0.05)
       plt.colorbar(im, cax=cax, ticks=np.arange(n_classes))

   if save_path:
       plt.savefig(save_path, bbox_inches='tight', dpi=600)

plot_datasets('Visualizations/gt_labels.png')

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def visualize_hyperspectral_with_patch(data, patch_size=7):
    # Create figure and main plot
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot the main hyperspectral image
    main_img = ax.imshow(np.mean(data, axis=2), cmap='viridis')
    
    # Remove axis labels and ticks
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Calculate point in top left quadrant for patch
    center_y = data.shape[0] // 4  # 1/4 of the height
    center_x = data.shape[1] // 4  # 1/4 of the width
    half_patch = patch_size // 2
    
    # Extract patch
    patch = data[
        center_y-half_patch:center_y+half_patch+1,
        center_x-half_patch:center_x+half_patch+1,
        :
    ]
    
    # Create inset axes
    axins = inset_axes(ax,
        width="30%",
        height="30%",
        loc='lower right')
    
    # Plot patch
    patch_img = axins.imshow(np.mean(patch, axis=2), cmap='viridis')
    axins.set_xticks([])
    axins.set_yticks([])
    
    # Draw box around selected area in main plot
    rect = plt.Rectangle(
        (center_x-half_patch, center_y-half_patch),
        patch_size, patch_size,
        fill=False,
        color='red',
        linewidth=2
    )
    ax.add_patch(rect)
    
    plt.show()


# Usage
data, gt = load_hyperspectral_data('Indian_pines')
visualize_hyperspectral_with_patch(data)
plt.savefig('Visualizations/cube_vis.png', bbox_inches='tight', dpi=300)

In [ ]:
data, ground_truth = load_hyperspectral_data('Salinas')
data.shape

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

with open('results.json', 'r') as file:
    data = json.load(file)

plt.style.use('ggplot')
colors = ['#1f77b4', '#ff7f0e']

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Adversarial Accuracy Across Different Epsilon Values', fontsize=16, y=1.05)

epsilon_values = [0.01, 0.03, 0.06, 0.09, 0.15]

# Create empty lines for the legend
dummy_lines = [
    plt.Line2D([0], [0], color=colors[0], marker='x', linestyle='-', markersize=8, linewidth=2, label='With Adversarial Training'),
    plt.Line2D([0], [0], color=colors[1], marker='x', linestyle='-', markersize=8, linewidth=2, label='Without Adversarial Training')
]

for idx, dataset in enumerate(data['model_performance'].keys()):
    adv_acc_with = []
    adv_acc_without = []
    
    for eps in epsilon_values:
        eps_key = f"epsilon_{eps:.2f}"
        if eps_key in data['model_performance'][dataset]:
            metrics_with = data['model_performance'][dataset][eps_key].get('with_adversarial_loss', {})
            metrics_without = data['model_performance'][dataset][eps_key].get('without_adversarial_loss', {})
            
            if metrics_with:
                adv_acc_with.append(np.mean(metrics_with['adversarial_accuracy']))
            if metrics_without:
                adv_acc_without.append(np.mean(metrics_without['adversarial_accuracy']))
    
    ax = axes[idx]
    ax.plot(epsilon_values, adv_acc_with, '-x', color=colors[0], markersize=8, linewidth=2)
    ax.plot(epsilon_values, adv_acc_without, '-x', color=colors[1], markersize=8, linewidth=2)
    
    ax.set_xlabel('Epsilon (ε)', fontsize=12)
    ax.set_ylabel('Adversarial Accuracy', fontsize=12)
    ax.set_title(f'{dataset.replace("_", " ").title()} Dataset', fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0.5, 1.0)
    ax.tick_params(axis='both', which='major', labelsize=10)

# Add single legend using dummy lines
fig.legend(handles=dummy_lines, bbox_to_anchor=(1.02, 0.5), loc='center left', borderaxespad=0, fontsize=12)

plt.tight_layout()
plt.savefig('adversarial_performance.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
with open('results.json', 'r') as file:
    data = json.load(file)

plt.style.use('ggplot')
colors = ['#1f77b4', '#ff7f0e']
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Robustness Score Comparison Across Different Epsilon Values', fontsize=16, y=1.05)

epsilon_values = [0.01, 0.03, 0.06, 0.09, 0.15]
dummy_lines = [
    plt.Line2D([0], [0], color=colors[0], marker='x', linestyle='-', markersize=8, linewidth=2, label='With Adversarial Training'),
    plt.Line2D([0], [0], color=colors[1], marker='x', linestyle='-', markersize=8, linewidth=2, label='Without Adversarial Training')
]

for idx, dataset in enumerate(data['model_performance'].keys()):
    rob_score_with = []
    rob_score_without = []
    for eps in epsilon_values:
        eps_key = f"epsilon_{eps:.2f}"
        if eps_key in data['model_performance'][dataset]:
            metrics_with = data['model_performance'][dataset][eps_key].get('with_adversarial_loss', {})
            metrics_without = data['model_performance'][dataset][eps_key].get('without_adversarial_loss', {})
            if metrics_with:
                rob_score_with.append(np.mean(metrics_with['robustness_score']))
            if metrics_without:
                rob_score_without.append(np.mean(metrics_without['robustness_score']))
    
    ax = axes[idx]
    ax.plot(epsilon_values, rob_score_with, '-x', color=colors[0], markersize=8, linewidth=2)
    ax.plot(epsilon_values, rob_score_without, '-x', color=colors[1], markersize=8, linewidth=2)
    ax.set_xlabel('Epsilon (ε)', fontsize=12)
    ax.set_ylabel('Robustness Score', fontsize=12)
    ax.set_title(f'{dataset.replace("_", " ").title()} Dataset', fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0.5, 1.0)
    ax.tick_params(axis='both', which='major', labelsize=10)
    
    # Add legend to the first subplot only
    if idx == 0:  # Pavia dataset plot
        ax.legend(handles=dummy_lines, 
                 loc='lower left',  # Position in lower left corner
                 fontsize=10,
                 framealpha=0.9)    # Make legend background slightly transparent

plt.tight_layout()
plt.savefig('robustness_comparison.png', bbox_inches='tight', dpi=300)
plt.show()

In [16]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Function to calculate mean of list values
def get_mean(values):
    return np.mean(values)

def create_performance_table(json_data):
    # Initialize lists to store data
    rows = []
    
    # Process each dataset
    for dataset in ['pavia', 'salinas', 'indian_pines']:
        for epsilon in ['0.01', '0.03', '0.06', '0.09', '0.15']:
            epsilon_key = f'epsilon_{epsilon}'
            
            # Get data for both training methods
            standard = json_data['model_performance'][dataset][epsilon_key]['without_adversarial_loss']
            adversarial = json_data['model_performance'][dataset][epsilon_key]['with_adversarial_loss']
            
            # Calculate means
            row = {
                'Dataset': dataset.replace('_', ' ').title(),
                'ε': epsilon,
                'Clean Acc. (Std)': f"{get_mean(standard['clean_accuracy']):.3f}",
                'Adv. Acc. (Std)': f"{get_mean(standard['adversarial_accuracy']):.3f}",
                'Rob. Score (Std)': f"{get_mean(standard['robustness_score']):.3f}",
                'Clean Acc. (Adv)': f"{get_mean(adversarial['clean_accuracy']):.3f}",
                'Adv. Acc. (Adv)': f"{get_mean(adversarial['adversarial_accuracy']):.3f}",
                'Rob. Score (Adv)': f"{get_mean(adversarial['robustness_score']):.3f}"
            }
            rows.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(rows)
    
    # Create figure and axis with appropriate size
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Hide axes
    ax.axis('tight')
    ax.axis('off')
    
    # Create table
    table = ax.table(
        cellText=df.values,
        colLabels=df.columns,
        cellLoc='center',
        loc='center',
        colColours=['#E6E6E6'] * len(df.columns)
    )
    
    # Style the table
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 1.5)
    
    # Adjust cell colors for better readability
    for i in range(len(df)):
        for j in range(len(df.columns)):
            cell = table[(i + 1, j)]
            if i % 5 == 0:  # Add subtle background for dataset separation
                cell.set_facecolor('#F5F5F5')
            else:
                cell.set_facecolor('white')
    
    # Add title
    plt.title('Model Performance Across Datasets and Training Methods', pad=20, fontsize=12)
    
    # Adjust layout and save
    plt.tight_layout()
    plt.savefig('model_performance_table.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()

# Load and process the JSON data
with open('results.json', 'r') as file:
   data = json.load(file)

create_performance_table(data)